In [1]:
import pandas as pd
#import numpy as np
import os
#import sys
#import matplotlib.pyplot as plt
#import matplotlib
from selenium import webdriver
# from selenium.webdriver.support.ui import Select
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
import random
import time
from datetime import datetime

#### 0. 환경설정

In [16]:
delay = 1.5   # 딜레이 시간

#### 1. 초기화

In [2]:
driver = webdriver.Chrome('chromedriver')

In [3]:
# result 폴더 생성
if not any([s == 'result' for s in os.listdir('.')]):
    os.mkdir('result')
    
# source 폴더 생성
if not any([s == 'source' for s in os.listdir('.')]):
    os.mkdir('source')

In [15]:
# 접속하기
url = 'http://www.encar.com/ev/ev_carsearchlist.do?carType=ev&searchType=model&TG.R=D#!%7B%22action%22%3A%22(And.Hidden.N._.CarType.A._.FuelType.%EC%A0%84%EA%B8%B0._.(C.GreenType.Y._.EvType.%EC%A0%84%EA%B8%B0%EC%B0%A8.))%22%2C%22toggle%22%3A%7B%7D%2C%22layer%22%3A%22%22%2C%22sort%22%3A%22ModifiedDate%22%2C%22page%22%3A1%2C%22limit%22%3A%2250%22%7D'
driver.get(url)

# [20개씩 보기] → [50개씩 보기]로 변환
viewer = Select(driver.find_element_by_css_selector('select#pagerow'))
viewer.select_by_value('50')
time.sleep(delay)

# 데이터 url 수집
batch = []
page = 1
while(True):
    try:
        driver.find_element_by_css_selector('div#pagination').find_element_by_xpath('//a[@data-page="{}"]'.format(page)).click()
    except:
        break
    time.sleep(delay)
    
    rows = driver.find_element_by_css_selector('tbody#sr_normal').find_elements_by_tag_name('tr')
    for row in rows:
        car_url = row.find_element_by_css_selector('td.inf > a').get_attribute('href')
        batch.append(car_url)
    page += 1

# url 저장
now = datetime.now().strftime('%Y%m%d%H%M%S')

# 결과 저장하기
df = pd.DataFrame(batch)
writer = pd.ExcelWriter('./result/car_url_{}.xlsx'.format(now), 'xlsxwriter')
df.to_excel(writer, index=False)
writer.save()
writer.close()

#### 2. page 저장하기

In [4]:
# URL 데이터 불러오기
df = pd.read_excel('./result/car_url_20190903162242.xlsx')

# 데이터 저장하기
for i, url in df.iterrows():
    driver.get(url[0])
    time.sleep(2 + 3*random.random())
    with open('./source/car_{}.html'.format(i+1), 'w', -1, encoding='utf-8') as f:
        f.write(driver.page_source)